# Scratchwork

In [1]:
from finite_algebras import *
from cayley_table import *
from permutations import *

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  15 example algebras are available.
  Use "get_example(INDEX)" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-group

## Subalgebras of a Semigroup

In [4]:
sg = ex.get_example(11)
sg.about()


Semigroup: Example 1.4.1
Instance ID: 140596974097552
Description: See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
Order: 6
Elements: ['a', 'b', 'c', 'd', 'e', 'f']
Identity: None
Associative? Yes
Commutative? No
Has Inverses? No
Cayley Table (showing indices):
[[0, 3, 0, 3, 0, 3],
 [1, 4, 1, 4, 1, 4],
 [2, 5, 2, 5, 2, 5],
 [3, 0, 3, 0, 3, 0],
 [4, 1, 4, 1, 4, 1],
 [5, 2, 5, 2, 5, 2]]


In [5]:
alg = sg
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(sg, partitions)


Subalgebras of <Semigroup:Example 1.4.1, ID:140596974097552>
  There are 4 unique subalgebras, up to isomorphisms, out of 10 total subalgebras
  as shown by the partitions below:

3 Isomorphic Semigroups of order 2:
      Example 1.4.1_subalgebra_0: ['c', 'e']
      Example 1.4.1_subalgebra_1: ['a', 'c']
      Example 1.4.1_subalgebra_8: ['a', 'e']

3 Isomorphic Commutative Groups of order 2:
      Example 1.4.1_subalgebra_2: ['b', 'e'] with identity 'e'
      Example 1.4.1_subalgebra_3: ['c', 'f'] with identity 'c'
      Example 1.4.1_subalgebra_9: ['a', 'd'] with identity 'a'

3 Isomorphic Semigroups of order 4:
      Example 1.4.1_subalgebra_4: ['a', 'c', 'd', 'f']
      Example 1.4.1_subalgebra_5: ['a', 'b', 'd', 'e']
      Example 1.4.1_subalgebra_7: ['b', 'c', 'e', 'f']

1 Semigroup of order 3:
      Example 1.4.1_subalgebra_6: ['a', 'c', 'e']



## Subgroups of an Alternating Group

In [6]:
a4 = ex.get_example(0)
a4.about()


Group: A4
Instance ID: 140596973906000
Description: Alternating group on 4 letters (AKA Tetrahedral group)
Order: 12
Identity: ()
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0      ()      ()       1
      1 (1 3 2) (1 2 3)       3
      2 (1 2 3) (1 3 2)       3
      3 (0 1)(2 3) (0 1)(2 3)       2
      4 (0 1 2) (0 2 1)       3
      5 (0 1 3) (0 3 1)       3
      6 (0 3 1) (0 1 3)       3
      7 (0 3 2) (0 2 3)       3
      8 (0 3)(1 2) (0 3)(1 2)       2
      9 (0 2 3) (0 3 2)       3
     10 (0 2 1) (0 1 2)       3
     11 (0 2)(1 3) (0 2)(1 3)       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 0, 6, 7, 8, 10, 11, 9, 5, 3, 4],
 [2, 0, 1, 10, 11, 9, 3, 4, 5, 8, 6, 7],
 [3, 4, 5, 0, 1, 2, 9, 10, 11, 6, 7, 8],
 [4, 5, 3, 9, 10, 11, 7, 8, 6, 2, 0, 1],
 [5, 3, 4, 7, 8, 6, 0, 1, 2, 11, 9, 10],
 [6, 7, 8, 1, 2, 0, 5, 3, 4, 10, 11, 9],
 [7, 8, 6, 5, 3, 4, 11, 9, 10, 0, 1, 2],
 [8, 6, 7, 11, 9, 10, 1, 2, 0, 

In [7]:
alg = a4
alg_subs = alg.proper_subalgebras(divisors_only=False, include_inverses=False)
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:A4, ID:140596973906000>
  There are 3 unique subalgebras, up to isomorphisms, out of 8 total subalgebras
  as shown by the partitions below:

3 Isomorphic Commutative Groups of order 2 with identity '()':
      A4_subalgebra_0: ['()', '(0 1)(2 3)']
      A4_subalgebra_1: ['()', '(0 2)(1 3)']
      A4_subalgebra_4: ['()', '(0 3)(1 2)']

4 Isomorphic Commutative Groups of order 3 with identity '()':
      A4_subalgebra_2: ['()', '(1 3 2)', '(1 2 3)']
      A4_subalgebra_3: ['()', '(0 3 2)', '(0 2 3)']
      A4_subalgebra_5: ['()', '(0 1 3)', '(0 3 1)']
      A4_subalgebra_7: ['()', '(0 1 2)', '(0 2 1)']

1 Commutative Normal Group of order 4 with identity '()':
      A4_subalgebra_6: ['()', '(0 1)(2 3)', '(0 3)(1 2)', '(0 2)(1 3)']



In [8]:
f4 = ex.get_example(9)
f4.about()


Field: F4
Instance ID: 140596974088144
Description: Field with 4 elements (from Wikipedia)
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       1       1       2
      2       a       a       2
      3     1+a     1+a       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: 1
Mult. Commutative? Yes
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 1, 2, 3], [0, 2, 3, 1], [0, 3, 1, 2]]


In [9]:
alg = f4
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Field:F4, ID:140596974088144>
  There are 1 unique subalgebras, up to isomorphisms, out of 3 total subalgebras
  as shown by the partitions below:

3 Isomorphic Commutative Normal Groups of order 2 with identity '0':
      F4_subalgebra_0: ['0', '1']
      F4_subalgebra_1: ['0', '1+a']
      F4_subalgebra_2: ['0', 'a']



In [10]:
ex6 = ex.get_example(12)
ex6.about()


Ring: Ex6
Instance ID: 140596974097744
Description: Example 6: http://www-groups.mcs.st-andrews.ac.uk/~john/MT4517/Lectures/L3.html
Order: 4
Identity: 0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0       0       0       1
      1       a       a       2
      2       b       b       2
      3       c       c       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
Mult. Identity: None
Mult. Commutative? No
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0], [0, 0, 1, 1], [0, 0, 2, 2], [0, 0, 3, 3]]


In [11]:
alg = ex6
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Ring:Ex6, ID:140596974097744>
  There are 1 unique subalgebras, up to isomorphisms, out of 3 total subalgebras
  as shown by the partitions below:

3 Isomorphic Commutative Normal Groups of order 2 with identity '0':
      Ex6_subalgebra_0: ['0', 'b']
      Ex6_subalgebra_1: ['0', 'c']
      Ex6_subalgebra_2: ['0', 'a']



In [12]:
psr = generate_powerset_group(4)
psr.about(max_size=16)


Group: PS4
Instance ID: 140596426412368
Description: Autogenerated Group on the powerset of 4 elements, with symmetric difference operator
Order: 16
Identity: {}
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4     {3}     {3}       2
      5  {0, 1}  {0, 1}       2
      6  {0, 2}  {0, 2}       2
      7  {0, 3}  {0, 3}       2
      8  {1, 2}  {1, 2}       2
      9  {1, 3}  {1, 3}       2
     10  {2, 3}  {2, 3}       2
     11 {0, 1, 2} {0, 1, 2}       2
     12 {0, 1, 3} {0, 1, 3}       2
     13 {0, 2, 3} {0, 2, 3}       2
     14 {1, 2, 3} {1, 2, 3}       2
     15 {0, 1, 2, 3} {0, 1, 2, 3}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 5, 6, 7, 2, 3, 4, 11, 12, 13, 8, 9, 10, 15, 14],
 [2, 5, 0, 8, 9, 1, 11, 12, 3, 4, 14, 6, 7, 15, 10, 13],
 [3, 6, 8, 0, 10, 1

In [13]:
alg = psr
%time alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)

CPU times: user 7.23 s, sys: 20.8 ms, total: 7.25 s
Wall time: 7.25 s

Subalgebras of <Group:PS4, ID:140596426412368>
  There are 3 unique subalgebras, up to isomorphisms, out of 65 total subalgebras
  as shown by the partitions below:

35 Isomorphic Commutative Normal Groups of order 4 with identity '{}':
      PS4_subalgebra_0: ['{}', '{0}', '{3}', '{0, 3}']
      PS4_subalgebra_1: ['{}', '{2, 3}', '{0, 1, 2}', '{0, 1, 3}']
      PS4_subalgebra_2: ['{}', '{0}', '{2}', '{0, 2}']
      PS4_subalgebra_3: ['{}', '{1}', '{2, 3}', '{1, 2, 3}']
      PS4_subalgebra_4: ['{}', '{3}', '{1, 2}', '{1, 2, 3}']
      PS4_subalgebra_7: ['{}', '{1}', '{2}', '{1, 2}']
      PS4_subalgebra_10: ['{}', '{0}', '{1}', '{0, 1}']
      PS4_subalgebra_12: ['{}', '{0}', '{1, 2, 3}', '{0, 1, 2, 3}']
      PS4_subalgebra_13: ['{}', '{0, 3}', '{0, 1, 2}', '{1, 2, 3}']
      PS4_subalgebra_16: ['{}', '{0, 1}', '{0, 2}', '{1, 2}']
      PS4_subalgebra_17: ['{}', '{1, 2}', '{0, 1, 3}', '{0, 2, 3}']
      PS4_subalg

In [14]:
# %time foo1 = psr.closed_subsets_of_elements(False)

In [15]:
# {len(x) for x in foo1}

In [16]:
# %time foo2 = psr.closed_subsets_of_elements(True)

In [17]:
# {len(x) for x in foo2}

In [18]:
# %time psr_subs = psr.unique_proper_subgroups()

In [19]:
# [psr_sub.order for psr_sub in psr_subs]

In [20]:
q8 = ex.get_example(13)
q8.about()


Group: Q8
Instance ID: 140596974098640
Description: Quaternion Group
Order: 8
Identity: 1
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0       1       1       1
      1       i      -i       4
      2      -1      -1       2
      3      -i       i       4
      4       j      -j       4
      5       k      -k       4
      6      -j       j       4
      7      -k       k       4
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 0, 7, 4, 5, 6],
 [2, 3, 0, 1, 6, 7, 4, 5],
 [3, 0, 1, 2, 5, 6, 7, 4],
 [4, 5, 6, 7, 2, 3, 0, 1],
 [5, 6, 7, 4, 1, 2, 3, 0],
 [6, 7, 4, 5, 0, 1, 2, 3],
 [7, 4, 5, 6, 3, 0, 1, 2]]


In [21]:
alg = q8
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:Q8, ID:140596974098640>
  There are 2 unique subalgebras, up to isomorphisms, out of 4 total subalgebras
  as shown by the partitions below:

1 Commutative Normal Group of order 2 with identity '1':
      Q8_subalgebra_0: ['1', '-1']

3 Isomorphic Commutative Normal Groups of order 4 with identity '1':
      Q8_subalgebra_1: ['1', '-1', 'k', '-k']
      Q8_subalgebra_2: ['1', '-1', 'j', '-j']
      Q8_subalgebra_3: ['1', 'i', '-1', '-i']



In [22]:
sd16 = ex.get_example(14)
sd16.about(max_size=16)


Group: SD16
Instance ID: 140596974099152
Description: Semidihedral group of order 16
Order: 16
Identity: e
Associative? Yes
Commutative? No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       t       t       2
      2       s  ts^2ts       8
      3      st   s^2ts       4
      4     s^2   ts^2t       4
      5    s^2t    s^2t       2
      6     tst  tsts^2       8
      7      ts   sts^2       4
      8    tsts    tsts       2
      9     sts     sts       2
     10  tsts^2     tst       8
     11   s^2ts      st       4
     12   ts^2t     s^2       4
     13    ts^2    ts^2       2
     14  ts^2ts       s       8
     15   sts^2      ts       4
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [1, 0, 7, 6, 13, 12, 3, 2, 9, 8, 15, 14, 5, 4, 11, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1],
 [3, 2, 9, 8, 15, 14, 5, 4, 11, 10, 1, 0, 7, 6, 13, 12],
 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

In [23]:
alg = sd16
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Group:SD16, ID:140596974099152>
  There are 6 unique subalgebras, up to isomorphisms, out of 13 total subalgebras
  as shown by the partitions below:

3 Isomorphic Commutative Groups of order 4 with identity 'e':
      SD16_subalgebra_0: ['e', 'ts', 'tsts', 'sts^2']
      SD16_subalgebra_10: ['e', 'st', 'tsts', 's^2ts']
      SD16_subalgebra_12: ['e', 's^2', 'tsts', 'ts^2t']

2 Isomorphic Commutative Groups of order 4 with identity 'e':
      SD16_subalgebra_1: ['e', 't', 'tsts', 'sts']
      SD16_subalgebra_2: ['e', 's^2t', 'tsts', 'ts^2']

5 Isomorphic Commutative Groups of order 2 with identity 'e':
      SD16_subalgebra_3: ['e', 'sts']
      SD16_subalgebra_4: ['e', 'ts^2']
      SD16_subalgebra_5: ['e', 'tsts']
      SD16_subalgebra_6: ['e', 't']
      SD16_subalgebra_7: ['e', 's^2t']

1 Normal Group of order 8 with identity 'e':
      SD16_subalgebra_8: ['e', 'st', 's^2', 'ts', 'tsts', 's^2ts', 'ts^2t', 'sts^2']

1 Normal Group of order 8 with identity 'e':
     

In [24]:
def ring_closure(alg, subset_of_elements):

    result = set(subset_of_elements)

    # Include additive inverses
    for elem in subset_of_elements:
        result.add(alg.inv(elem))

    # Add the sum of all possible pairs
    for pair in it.product(result, result):
        result.add(alg.add(*pair))
        
    # Add the product of all possible pairs
    for pair in it.product(result, result):
        result.add(alg.mult(*pair))
        
    # If the input set of elements increased, recurse ...
    if len(result) > len(subset_of_elements):
        return ring_closure(alg, list(result))

    # ...otherwise, stop and return the result
    else:
        return list(result)

In [40]:
alg1 = generate_algebra_mod_n(12)
alg1.about(use_table_names=True)


Ring: R12
Instance ID: 140596976092112
Description: Autogenerated Ring of integers mod 12
Order: 12
Identity: a0
Associative? Yes
Commutative? Yes
Elements:
   Index   Name   Inverse  Order
      0      a0      a0       1
      1      a1     a11      12
      2      a2     a10       6
      3      a3      a9       4
      4      a4      a8       3
      5      a5      a7      12
      6      a6      a6       2
      7      a7      a5      12
      8      a8      a4       3
      9      a9      a3       4
     10     a10      a2       6
     11     a11      a1      12
Cayley Table (showing names):
[['a0', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11'],
 ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a0'],
 ['a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a0', 'a1'],
 ['a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a0', 'a1', 'a2'],
 ['a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a0', 'a1', 'a2', 'a3'],
 ['a5', 'a

In [42]:
alg = alg1
alg_subs = alg.proper_subalgebras()
partitions = partition_into_isomorphic_lists(alg_subs)
about_isomorphic_partitions(alg, partitions)


Subalgebras of <Ring:R12, ID:140596976092112>
  There are 4 unique subalgebras, up to isomorphisms, out of 4 total subalgebras
  as shown by the partitions below:

1 Commutative Normal Group of order 4 with identity 'a0':
      R12_subalgebra_0: ['a0', 'a3', 'a6', 'a9']

1 Commutative Normal Group of order 6 with identity 'a0':
      R12_subalgebra_1: ['a0', 'a2', 'a4', 'a6', 'a8', 'a10']

1 Commutative Normal Group of order 2 with identity 'a0':
      R12_subalgebra_2: ['a0', 'a6']

1 Commutative Normal Group of order 3 with identity 'a0':
      R12_subalgebra_3: ['a0', 'a4', 'a8']



In [52]:
elems = ['a0', 'a4']

In [53]:
alg1.closure(elems, True)

['a8', 'a4', 'a0']

In [54]:
ring_closure(alg1, elems)

['a8', 'a4', 'a0']